<a href="https://colab.research.google.com/github/Matteo-Candi/ADM_HW3/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Homework 3 - Places of the world**
---

In [4]:
import requests
import bs4
from bs4 import BeautifulSoup as bs
from tqdm import tqdm
import os
import pandas as pd
import numpy as np
from datetime import datetime
import json
import random as rdm
from functools import reduce

## **1. Data collection**

### 1.1. Get the list of places

In [ ]:
with open('Places_url.txt', 'x') as f:
    for i in tqdm(range(1,401)):             
        url = f'https://www.atlasobscura.com/places?page={i}&sort=likes_count'
        result = requests.get(url)
        soup = bs(result.text)
        matches = soup.find_all("a", {"class": "content-card content-card-place"})
        for match in matches:
            f.write(match['href'] + '\n')

### 1.2. Crawl places

In [14]:
os.mkdir(f'Locations')
places_url = pd.read_csv(os.getcwd()+'/Places_url.txt', header=None)


for page in tqdm(range(0,400)):
    os.mkdir(f'Locations/page_{page + 1}')
    for loc in range(0,18):
        with open(os.getcwd()+'/Locations/page_'+str(page+1)+'/location_'+str(18*page+loc+1)+'.html', 'w', encoding="utf-8") as f:
            url = 'https://www.atlasobscura.com' + places_url.iloc[(18 * page) + loc][0]
            result = requests.get(url)
            html = result.text
            f.write(html)

100%|██████████| 9/9 [01:33<00:00, 10.44s/it]


### 1.3 Parse downloaded pages

In [63]:
# Get all the data for one place in a dataframe
# input : String corresponding to the html_file
# output : dataframe containing all the data scraped 
def get_data_for_one_place(html_file):

    soup = bs(html_file)

    df = pd.DataFrame(columns=['placeName','placeTags','numPeopleVisited','numPeopleWant','placeDesc','placeShortDesc','placeNearby','placeAddress','placeAlt','placeLong','placeEditors','placePubDate','placeRelatedLists','placeRelatedPlaces','placeURL']) 

    placeName = soup.find('h1', {'class':'DDPage__header-title'})
    if placeName is not None :
        df['placeName'] = [placeName.text]
    else :
        df['placeName'] = ''

    placeTags = soup.find('div', {'class':'DDPage__header-place-location'})
    if placeTags is not None :
        df['placeTags'] = placeTags.text
    else :
        df['placeTags'] = ''

    L = soup.find_all('div', {'class':'col-xs-4X js-submit-wrap js-been-to-top-wrap action-btn-col hidden-print'})
    if L is not None and len(L)!=0 and L[0].find('div', {'class':'title-md item-action-count'}) is not None :
        df['numPeopleVisited'] = int(L[0].find('div', {'class':'title-md item-action-count'}).text)
    else :
        df['numPeopleVisited'] = ''

    L = soup.find_all('div', {'class':'col-xs-4X js-submit-wrap js-like-top-wrap action-btn-col hidden-print'})  
    if L is not None and len(L)!=0 and L[0].find('div', {'class':'title-md item-action-count'}) is not None :
        df['numPeopleWant'] = int(L[0].find('div', {'class':'title-md item-action-count'}).text)
    else :
        df['numPeopleWant'] = ''

    placeDesc = soup.find('div', {'id':'place-body'})
    if placeDesc is not None :
        df['placeDesc'] = placeDesc.text.replace('\n',' ').replace(u'\xa0',u' ')
    else :
        df['placeDesc'] = ''
    
    placeShortDesc = soup.find('h3', {'class': 'DDPage__header-dek'})
    if placeShortDesc is not None :
        df['placeShortDesc'] = placeShortDesc.text.replace('\n',' ').replace(u'\xa0',u' ')
    else :
        df['placeShortDesc'] = ''

    placeNearby = set([match.text for match in soup.find_all('div', {'class': 'DDPageSiderailRecirc__item-title'})])
    if len(placeNearby) != 0 :
        df['placeNearby'] = ','.join(list(placeNearby))
    else :
        df['placeNearby'] = ''

    match_address = soup.find("address", {"class":"DDPageSiderail__address"})
    if match_address is not None :
        placeAddress = match_address.find('div').contents[0:5:2]
        df['placeAddress'] = "".join(placeAddress).replace('\n','')
    else :
        df['placeAddress'] = ''

    match = soup.find("div", {"class":"DDPageSiderail__coordinates js-copy-coordinates"})
    if match is not None :
        match = match.get_text().split()
        df['placeAlt'] = float(match[0][:-1])
        df['placeLong'] = float(match[1])
    else :
        df['placeAlt'] = ''
        df['placeLong'] = ''

    contributors = []
    for contributor in soup.find_all('div', {'class':'DDPContributors'})[0].find_all('a', {'class':'DDPContributorsList__contributor'}) :
        if contributor.find('span') is not None :
            contributors.append(contributor.find('span').text)
        else :
            contributors.append(contributor.text)
    if len(contributors) != 0 :
        df['placeEditors'] = ','.join(list(set(contributors)))
    else :
        df['placeEditors'] = ''

    placePubDate = soup.find("div", {"class":"DDPContributor__name"})
    if placePubDate is not None :
        df['placePubDate'] = datetime.strptime(placePubDate.text, '%B %d, %Y')
    else :
        df['placePubDate'] = ''

    placeRelatedLists = []
    placeRelatedPlaces = []
    for container in soup.find_all('div', {'class' : 'full-width-container CardRecircSection'}) :
        title = container.find('div', {'class':'CardRecircSection__title'}).text
        if 'Appears' in title :
            for container_related in container.find_all('h3', {'class':'Card__heading --content-card-v2-title js-title-content'}) :
                placeRelatedLists.append(container_related.text[1:-1])
        elif 'Related Places' in title :
            for container_related in container.find_all('h3', {'class':'Card__heading --content-card-v2-title js-title-content'}) :
                placeRelatedPlaces.append(container_related.text[1:-1])
    if len(placeRelatedLists) != 0 :
        df['placeRelatedLists'] = ','.join(placeRelatedLists)
    else :
        df['placeRelatedLists'] = ''
    if len(placeRelatedPlaces) != 0 :
        df['placeRelatedPlaces'] = ','.join(placeRelatedPlaces)
    else :
        df['placeRelatedPlaces'] = ''

    if len(soup.find_all("link", {'rel':'canonical'})) != 0 :
        df['placeURL'] = soup.find_all("link", {'rel':'canonical'})[0]['href']   
    else :
        df['placeURL'] = ''  
        
    return df


In [67]:
# We get all the html files contained in the Locations directory
# For each html file we call the get_data_for_one_place function
# We upload the output of the function in a tsv file

for page in tqdm(range(0,400)):
    for loc in range(0,18):
        with open(os.getcwd()+'/Locations/page_'+str(page+1)+'/location_'+str(18*page+loc+1)+'.html', 'r', encoding="utf-8") as f :
            html_file = f.read()
        df = get_data_for_one_place(html_file)
        df.to_csv(os.getcwd()+'/Locations/page_'+str(page+1)+'/location_'+str(18*page+loc+1)+'.tsv', sep='\t', index=False, header=None)


100%|██████████| 400/400 [28:34<00:00,  4.29s/it]


---
## **2. Search Engine**

In [69]:
import nltk
import re

nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')
stemmers = nltk.stem.SnowballStemmer('english')

def remove_punctuations(comment) :
    comment = re.sub(r'[? | ! | \' |" | #]','',comment)
    comment = re.sub(r'[. | , | ) | ( | \ | / ]','',comment)
    return comment

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\yharr\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [70]:
# Mettre les contenus de tous les fichiers tsv dans un seul dataframe
# Effectuer un preprocessing sur le champ long description de ce dataframe
# Enregistrer ce dataframe dans un fichier tsv à la racine
# We want to get the data of all the tsv files and put it in only one tsv file
# Before doing it we preprocess all the attributes placeDesc and placeShortDesc of each tsv file

def getPreProcessedData(description) :
    filtered_data = []
    for word in description.split() :
        # We remove the puncuations
        for clean_word in remove_punctuations(word).split() :
            # We check if we have only alpha characters and if the word has a len of 3 at least
            if clean_word.isalpha() and len(clean_word) > 2 :
                # We check if it's not a stopword
                if clean_word.lower() not in stopwords :
                    # We apply the stemmatization
                    stemmed_word = stemmers.stem(clean_word.lower())
                    filtered_data.append(stemmed_word)
                else :
                    continue
            else :
                continue
    return " ".join(filtered_data)


cols = columns=['placeName','placeTags','numPeopleVisited','numPeopleWant','placeDesc','placeShortDesc','placeNearby','placeAddress','placeAlt','placeLong','placeEditors','placePubDate','placeRelatedLists','placeRelatedPlaces','placeURL']
df = pd.DataFrame(columns=cols)
for page in tqdm(range(0,400)):
    for loc in range(0,18):
        df_tsv = pd.read_csv(os.getcwd()+'/Locations/page_'+str(page+1)+'/location_'+str(18*page+loc+1)+'.tsv', sep='\t', names=cols)
        df_tsv.placeDesc = df_tsv.placeDesc.apply(lambda row : getPreProcessedData(row))
        df_tsv.placeShortDesc = df_tsv.placeShortDesc.apply(lambda row : getPreProcessedData(row))
        df = pd.concat([df,df_tsv])

df.to_csv(os.getcwd()+'/locations_data.tsv', sep='\t', index=False)     


100%|██████████| 400/400 [01:43<00:00,  3.86it/s]


In [76]:
# Here is the full dataset of places with placeDesc and placeShortDesc preprocessed.
df_out = pd.read_csv(os.getcwd()+'/locations_data.tsv', sep='\t')
df_out

,placeName,placeTags,numPeopleVisited,numPeopleWant,placeDesc,placeShortDesc,placeNearby,placeAddress,placeAlt,placeLong,placeEditors,placePubDate,placeRelatedLists,placeRelatedPlaces,placeURL
0,City Hall Station,"Manhattan, New York",1828,8583,first new york citi subway built oper interbor...,beauti abandon new york subway station complet...,"Tunnel Number 3,African Burial Ground National...","31 Centre StNew York, New York, 10007United St...",40.7134,-74.0046,"Allan,erjeffery,fosterc827,offtrackplanet,wyth...",2010-05-08,30 Unexpected Places to Have a Joyful Adventur...,"Crystal Palace Subway,Moscow Metro Stations,Ro...",https://www.atlasobscura.com/places/city-hall-...
1,Highgate Cemetery,"London, England",2618,8182,open highgat one infam cemeteri origin open on...,london creepiest cemeteri site duel magician m...,"Parkland Walk,Dick Whittington’s Cat ,World's ...","Swain's Lane, HighgateLondon, England, N6Unite...",51.5675,-0.1483,"gingercinnamon,Annetta Black,lushjay,SEANETTA,...",2014-08-09,"The World's Top 100 Wonders in 2018,London's T...","Jewett City Vampires,Tomb of the Mather Family...",https://www.atlasobscura.com/places/highgate-c...
2,Leadenhall Market,"London, England",3135,7570,ornat paint roof cobbl floor leadenhal market ...,ornat victorian marketplac set diagon alley le...,"The Cornhill Devils ,Philpot Lane Mice Sculptu...","London, England, EC3VUnited Kingdom",51.5126,-0.0834,"Lyloueen,lili,Meg,amiedd,JZA,Gavin,raymondwinn...",2016-08-01,"The Ultimate Guide to Stunning, Surprising, or...","Rivendell,Bagdad Cafe,Gare de la Ciotat,Drvengrad",https://www.atlasobscura.com/places/leadenhall...
3,The Wave Organ,"San Francisco, California",2429,7408,locat jetti san francisco bay wave organ built...,huge music instrument play ocean,"Palace of Fine Arts,Long Now Orrery,The Stern ...","83 Marina Green DrSan Francisco, California, 9...",37.8085,-122.4401,"td007,Mark Casey,Chassy,SEANETTA,hana,Saal333,...",2008-11-21,Leonardo Nam's 16 Quirky Roadside Attractions ...,"Sea Organ,Silent Green Kulturquartier,St. John...",https://www.atlasobscura.com/places/wave-organ
4,Catacombes de Paris,"Paris, France",4447,7046,parisian polic assign train exercis previous u...,vast legendari catacomb hold secret much stran...,"Arago Medallions,Sculptures de Décure,Jeannot'...","1 Place Denfert-RochereauParis, 75014France",48.8343,2.3322,"ramonrodz2212,Fred Cherrygarden,ceasterling,Es...",2009-02-13,"19 Catacombs Sure to Tingle Your Spine,The Wor...","Ossario di San Martino,Leuk Charnel House,Sant...",https://www.atlasobscura.com/places/catacombes...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7195,The Cairo,"Washington, D.C.",342,405,washington residenti neighborhood typifi low r...,unaccept tall build real reason washington dcs...,"Annie's Paramount Steakhouse,House of the Temp...","1615 Q Street NWWashington, District of Columb...",38.9113,-77.0375,"Greg Jones,matthewbgilmore,Elliot Carter",2017-09-12,NaN,"145 Rue Lafayette,Architect of the Capitol Arc...",https://www.atlasobscura.com/places/the-cairo-...
7196,Winganon Space Capsule,"Talala, Oklahoma",120,405,drive along dull stretch road talala winganon ...,detach cement mixer transform resembl relic ob...,"Playtower,Totem Pole Park,Bowling Ball Yard Art","E 300 RoadTalala, OklahomaUnited States",36.5828,-95.6516,"5qmbcxnkwy,Darrell Powers,MNtraveler,CoolCrab,...",2017-08-14,NaN,"Folk Art Park,Apple Valley Hillbilly Garden an...",https://www.atlasobscura.com/places/winganon-s...
7197,Cook & Book,"Brussels, Belgium",124,405,cook book brussel far bookstor size supermarke...,wonder kitsch bookstor size supermarket,"Bibliotheca Wittockiana,Temple of Human Passio...",1 Place du Temps LibreWoluwe-Saint-LambertBrus...,50.8480,4.4373,"simonlitton,rhumphrey17,Producer Dani",2017-02-08,The Ultimate Guide to Wondrous Independent Boo...,"For Keeps Bookstore,Fantagraphics,La Llama,Whi...",https://www.atlasobscura.com/places/cook-book
7198,Xul Solar Museum,"Buenos Aires, Argen

### 2.1. Conjunctive query

 2.1.1) Create your index!

In [5]:
df = pd.read_csv('locations_data.tsv', sep='\t')
data = df.placeDesc

splitted_rows = data.apply(lambda row: row.split())
vocabulary = list(set([i for row in splitted_rows for i in set(row)]))
vocabulary.sort()

In [ ]:
inverted_index = {}
for i, word in tqdm(enumerate(vocabulary)):
    inverted_index[i] = list(splitted_rows[splitted_rows.apply(lambda row: word in row)].index)

2.1.2) Execute the query

In [8]:
query = 'american museum'
query = query.split()
stemmed_query = [stemmers.stem(word.lower()) for word in query]
id_query_words = [vocabulary.index(word) for word in stemmed_query]
docs = [inverted_index[str(id)] for id in id_query_words]

In [10]:
matches = list(reduce(lambda x,y: set(x).intersection(y), docs))           #  Supposing the query has only 2 words (for now)
matches.sort()
output = df[['placeName', 'placeDesc', 'placeURL']]
output.loc[matches[:10]]


,placeName,placeDesc,placeURL
59,The Witch House of Salem,salem witchcraft trial took place februari may...,https://www.atlasobscura.com/places/witch-hous...
141,Museum of the Weird,dime dime store museum account endang speci fi...,https://www.atlasobscura.com/places/museum-weird
173,National Atomic Testing Museum,las vega oasi desert one sleep everi vice imag...,https://www.atlasobscura.com/places/national-a...
224,Canyons of the Ancients,ripe quiet reflect simpli canyon ancient outdo...,https://www.atlasobscura.com/places/canyons-of...
320,The Natural Bridge,often cite place among great wonder natur worl...,https://www.atlasobscura.com/places/the-natura...
329,Uncommon Objects,like eleg antiqu mall gone horribl wrong diffe...,https://www.atlasobscura.com/places/uncommon-o...
356,KattenKabinet,death pet inspir number reaction rare anyon ta...,https://www.atlasobscura.com/places/kattenkabinet
360,CDC Museum,hit film outbreak resid unit state come terror...,https://www.atlasobscura.com/places/cdc-museum


### 2.2. Conjunctive query & Ranking score

2.2.1) Inverted index

2.2.2) Execute the query

---
## **3. Define a new score!**

---
## **4. Visualizing the most relevant places**

---
## **5. BONUS: More complex search engine**

---
## **6. Command line question**

---
## **7. Theoretical question**

In [ ]:
with open('ApplicantsInfo.txt', 'x') as f:
    url = f'https://adm2022.s3.amazonaws.com/ApplicantsInfo.txt'
    result = requests.get(url)
    soup = BeautifulSoup(result.text)
    s = soup.find_all('body')
    print(s)
    f.write(str(s))

In [ ]:
Application = pd.read_csv('ApplicantsInfo.txt', header=None)


In [ ]:
n =Application.iloc[0]
Application.drop(0, inplace=True)
N= int(n[0][10:15])
M= int(n[0][16:20])

In [ ]:
df_average = pd.DataFrame(columns=('names','surnames','averages'))
for i in range(N):
    name = Application.iloc[i][0].split(' ')[0]
    surname = Application.iloc[i][0].split(' ')[1]
    all_points = Application.iloc[i][0].split(' ')[2:]
    count = 0
    for i in all_points:
        s = int(i)
        count += s
    average = round(count/M,2)
    df_average.loc[i]=[name, surname, average]
    
df_average.head()
df_average.describe()  

In [ ]:
lista = []
for i in tqdm(range(N)):
    name = Application.iloc[i][0].split(' ')[0]
    surname = Application.iloc[i][0].split(' ')[1]
    all_points = Application.iloc[i][0].split(' ')[2:]
    count = 0
    for i in all_points:
        s = int(i)
        count += s
    average = round(count/M,2)
    lista.append((name, surname, average))


####  bubbleSort( )

In [ ]:
def BubbleSort(lista):
    for i in tqdm(range(N)):  #N
        for j in range(i):  #i
            if lista[j][2] >= lista[j+1][2]:  #1
                if lista[j][2] == lista[j+1][2]:  #1
                    if lista[j][0] >= lista[j+1][0]:  #1
                        if lista[j][0] == lista[j+1][0]:  #1
                            if lista[j][1] > lista[j+1][1]: #1 
                                lista[j],lista[j+1]=lista[j+1],lista[j]  #1
                        else:
                            lista[j],lista[j+1]=lista[j+1],lista[j]  #1
                else:
                    lista[j],lista[j+1]=lista[j+1],lista[j]  #1
    return lista
                

In [ ]:
BubbleSort(lista)

#### MergeSort( )

In [ ]:
def Merge(l1,l2):
    n=len(l1)
    m=len(l2)
    lista=[]
    i=0
    j=0
    while i < n and j < m:
        if l1[i][2] <= l2[j][2]:
            if l1[i][2] == l2[j][2]:
                if l1[i][0] <= l2[j][0]:
                    if l1[i][0] == l2[j][0]:
                        if l1[i][1] < l2[j][1]:
                            lista.append(l1[i])
                            i+=1
                        else:
                            lista.append(l2[j])
                            j+=1
                    else:
                        lista.append(l1[i])
                        i+=1
                else:
                    lista.append(l2[j])
                    j+=1
            else:
                lista.append(l1[i])
                i+=1
                
        else:
            lista.append(l2[j])
            j+=1

    while i < n:
            lista.append(l1[i])
            i += 1
    while j < m:
            lista.append(l2[j])
            j += 1
    return lista


In [ ]:
def MergeSort(lista):
    if len(lista) == 1:
        return lista
    else:
        lista1=lista[:len(lista)//2]
        lista2=lista[len(lista)//2:]
        l1=MergeSort(lista1)
        l2=MergeSort(lista2)
        return Merge(l1,l2)

In [ ]:
MergeSort(lista)

#### QuickSort( )

In [ ]:

def QuickSort(lista):
    if len(lista) <= 1:
        return lista
    else:
        pivot = rdm.choice(lista)
        lista.remove(pivot)
        l1=[]
        l2=[]
        for x in lista:
            if x[2] <= pivot[2]:
                if x[2] == pivot[2]:
                    if x[0] <= pivot[0]:
                        if x[0] == pivot[0]:
                            if x[1] < pivot[1]:
                                l1.append(x)
                            else:
                                l2.append(x)
                        else:
                            l1.append(x)
                    else:
                        l2.append(x)
                else:
                    l1.append(x)
            else:
                l2.append(x)
        l= QuickSort(l1)+[pivot]+ QuickSort(l2)
        return l
                

In [ ]:
QuickSort(lista)